**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random
import math
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Flatten,Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [15]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [26]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True,counter=1):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= (epsilon/counter):
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Epsilon is essential as it set the level of exploration of the agent. 
If we are in the training phase, the agent will, with a probability epsilon, perform a random action to explore the environment and learn new possibilities which will hopefully  help him to improve its overall score in the long term. With a probability 1-epsilon, the agent will exploit its known policies and try to get the best score from what it already knows.

***
### The Game

#The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)


epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
    

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [28]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [29]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=150 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board sets the random cheese and poisonous cells while the position sets the overall environment/ the overall game space. It sets poisonous cells at the edges to prevent the agent from getting out of the game space.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [30]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [76]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # No reinforcement strategy for a random agent
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [34]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/13.0. Average score (-3.0)
Win/lose count 12.5/16.0. Average score (-3.25)
Win/lose count 9.5/11.0. Average score (-2.6666666666666665)
Win/lose count 7.5/11.0. Average score (-2.875)
Win/lose count 10.0/11.0. Average score (-2.5)
Win/lose count 9.5/8.0. Average score (-1.8333333333333333)
Win/lose count 10.0/13.0. Average score (-2.0)
Win/lose count 10.0/12.0. Average score (-2.0)
Win/lose count 5.0/15.0. Average score (-2.888888888888889)
Win/lose count 9.0/20.0. Average score (-3.7)
Win/lose count 8.5/28.0. Average score (-5.136363636363637)
Win/lose count 14.0/20.0. Average score (-5.208333333333333)
Win/lose count 11.0/13.0. Average score (-4.961538461538462)
Win/lose count 11.0/18.0. Average score (-5.107142857142857)
Win/lose count 15.0/9.0. Average score (-4.366666666666666)
Win/lose count 7.0/7.0. Average score (-4.09375)
Win/lose count 8.0/16.0. Average score (-4.323529411764706)
Win/lose count 11.0/18.0. Average score (-4.472222222222222)
Win/lose count 1

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

{t=1}^{T}



Let us assume here that 
$T=\infty$ 
and \begin{equation*}{P_{ss'}^{a}} = p(s_{t+1}=s'|s_t = s, a_t = a )
\end{equation*}
1.
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]+ E_{p^{\pi}}[ r(s,a)] \> .
\end{equation*}

\begin{equation*}Q^\pi(s,a)= {\gamma}E_{p^{\pi}}[\sum_{t=0}^{T}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]+ E_{p^{\pi}}[ r(s,a)] \> .
\end{equation*}

\begin{equation*}Q^\pi(s,a)= {\gamma}{\sum_{s'}}[{P_{ss'}^{a}}E_{p^{\pi}}[\sum_{t=0}^{T}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s']]+ E_{p^{\pi}}[ r(s,a)] \> .
\end{equation*}

\begin{equation*}Q^\pi(s,a)= {\gamma}{\sum_{s'}}{\sum_{a'}}[{P_{ss'}^{a}}E_{p^{\pi}}[\sum_{t=0}^{T}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']]+ E_{p^{\pi}}[ r(s,a)] \> .
\end{equation*}

\begin{equation*}Q^\pi(s,a)= {\gamma}{\sum_{s'}}{\sum_{a'}}[{P_{ss'}^{a}}Q^{\pi}(s',a')\pi(s',a')]+ E_{p^{\pi}}[ r(s,a)] \> .
\end{equation*}

And we have:

\begin{equation*}E_{p^{\pi}}[ r(s,a)]= {\sum_{s'}}[{P_{ss'}^{a}}r(s,a)].
\end{equation*}


\begin{equation*}E_{p^{\pi}}[ r(s,a)]= {\sum_{s'}}{\sum_{a'}}[{P_{ss'}^{a}}r(s,a)\pi(s',a')].
\end{equation*}

Finally:

\begin{equation*}Q^\pi(s,a)= {\gamma}{\sum_{s'}}{\sum_{a'}}[{P_{ss'}^{a}}Q^{\pi}(s',a')\pi(s',a')]+ {\sum_{s'}}{\sum_{a'}}[{P_{ss'}^{a}}r(s,a)\pi(s',a')].
\end{equation*}

\begin{equation*}Q^\pi(s,a)= {\sum_{s'}}{\sum_{a'}}{P_{ss'}^{a}}\pi(s',a')[r(s,a)+{\gamma}Q^{\pi}(s',a')].
\end{equation*}

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

2.
\begin{equation*}
\max_{\pi} \sum_{s'} {P_{ss'}^{a}}Q^{\pi}(s',a') \leq \sum_{(s',a')} {P_{ss'}^{a}}\max_{\pi}Q^{\pi}(s',a') 
\end{equation*}

Let's set ${\pi^{*}}(s',a') = \arg\max_{\pi} Q^{\pi}(s',a')$


\begin{align*}
 \sum_{s'} {P_{ss'}^{a}} \max_{\pi}Q^{\pi}(s',a') &= \sum_{
 s'} {P_{ss'}^{a}} Q^{\pi^{*}(s',a')}(s',a') \ \leq \max_{\pi} \sum_{s'} {P_{ss'}^{a}}Q^{\pi}(s',a') 
\end{align*}

Finally :
\begin{align}
Q^{*}(s,a)&= r(s,a) + \gamma  \sum({P_{ss'}^{a}}\max_{a'}\max_{\pi}Q^{\pi}(s',a') \\
Q^{*}(s,a)&= r(s,a) + \gamma \sum({P_{ss'}^{a}}\max_{a'} Q^{*}(s',a') \\ 
Q^{*}(s,a)&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

3. 
From the equation in 2., we know that when the loss function converges to 0, it will in fact make the action value function converge to the optimal one.
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [35]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>self.max_memory:
            del self.memory[0]
            self.memory.append(m)
        else:
            self.memory.append(m)
        return self.memory

    def random_access(self):
        i = np.random.randint(0,len(self.memory),size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [39]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        counter = 0

        while not game_over:
            # The agent performs an action
            counter += counter
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [40]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__()
        super().set_epsilon(e = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        # Epsilon
        self.epsilon = epsilon

    def learned_act(self, s):
        # Select the action (the index) with best reward using Qt
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        # input_states = observations by the agent
        # store the results
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # choose a random previously stored result
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            # store the predicted rewards for the state s using the deep network
            target_q[i]= self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            if game_over_:
                ######## FILL IN
                # if game_over, no further action for this game, store the state
                input_states[i]=s_
                # update the reward of action a and state s with the known/ observed reward r_
                target_q[i,a_]=r_
                
            else:
                ######## FILL IN
                # this is not the end of the game, store the state
                input_states[i]=s_ 
                # update the reward of action a and state s with 2 components:
                # 1. the known/observed reward r_
                # 2. + add the discounted maximum of predicted rewards of the new state n_s (inspired from demonstration above)
                target_q[i,a_] = r_ + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        # retrain the model for next period 
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        input_model = Input(shape = (5,5,self.n_state))
        
        fully_layer_1 = Dense(4, activation="relu")(input_model)
        Reshape_1 = Reshape(target_shape=(100,))(fully_layer_1)
        fully_layer_2 = Dense(16, activation="relu")(Reshape_1)
        fully_layer_3 = Dense(8, activation="relu")(fully_layer_2 )
        fully_layer_4 = Dense(4, activation="softmax")(fully_layer_3)
        
        model = Model(inputs =[input_model] , outputs =[fully_layer_4])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [41]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/150 | Loss 0.0003 | Win/lose count 2.5/1.0 (1.5)
Epoch 001/150 | Loss 0.0120 | Win/lose count 5.0/9.0 (-4.0)
Epoch 002/150 | Loss 0.0002 | Win/lose count 3.0/4.0 (-1.0)
Epoch 003/150 | Loss 0.0062 | Win/lose count 3.0/1.0 (2.0)
Epoch 004/150 | Loss 0.0078 | Win/lose count 6.0/3.0 (3.0)
Epoch 005/150 | Loss 0.0086 | Win/lose count 2.5/6.0 (-3.5)


KeyboardInterrupt: 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [42]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
    
        input_model_2 =  Input(shape = (5,5,self.n_state))
        conv_1 = Conv2D(5,3,activation='relu')(input_model_2)
        conv_2 = Conv2D(5,3,activation='relu')(conv_1)
        flatten_1 = Flatten()(conv_2)
        connected_layer = Dense(units=4,activation="softmax")(flatten_1)
        model = Model(inputs =[input_model_2] , outputs =[connected_layer])
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/150 | Loss 0.0271 | Win/lose count 8.5/14.0 (-5.5)
Epoch 001/150 | Loss 0.0134 | Win/lose count 3.5/11.0 (-7.5)


KeyboardInterrupt: 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [ ]:
env = Environment(grid_size=size, max_time=T,temperature=0.4)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

The 2 models are not really performant as they tend not to explore the map. They repeat a fixed succession of actions so that they do not explore the map. Yet, it seems that the convolutional model performs a little bit better. Indeed, on average, it has better average scores than the fully connected model.
When increasing the temperature, the 2 models perform better (average scores increased) which is normal because there are less poisonous cells. The 2 models also tend to explore more the map. Yet, it is still not satisfying as the 2 models are blocked after a certain number of actions.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [70]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.4
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
        pass

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
        pass
    
def train_explore(agent,env,epoch,prefix=''):
        # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        state = env.reset()
        counter = 1
        
        game_over = False

        win = 0
        lose = 0
        #INTRODUCEA COUNTER TO HAVE A DECAYING EPSILON
        #SET IT INITIALLY TO ONE
        

        while not game_over:
            # CREATE A NEW PARAMETER FOR THE AGENT 
            # IT COUNTS THE NUMBER OF OBSERVATIONS OF THE AGENT
            # EPSILON WILL DECAY AS FOLLOWED: EPSILON/COUNTER
            action = agent.act(state,counter)
            counter+=1

           
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

In [71]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)

In [72]:
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/150 | Loss 0.0440 | Win/lose count 9.0/63.19999999999984 (-54.19999999999984)
Epoch 001/150 | Loss 0.0551 | Win/lose count 7.4/75.60000000000001 (-68.2)
Epoch 002/150 | Loss 0.0403 | Win/lose count 4.799999999999999/82.60000000000011 (-77.80000000000011)
Epoch 003/150 | Loss 0.0448 | Win/lose count 3.600000000000001/81.20000000000007 (-77.60000000000008)
Epoch 004/150 | Loss 0.0486 | Win/lose count 2.4000000000000004/82.20000000000012 (-79.80000000000011)
Epoch 005/150 | Loss 0.0417 | Win/lose count 1.7000000000000002/78.40000000000002 (-76.70000000000002)
Epoch 006/150 | Loss 0.0571 | Win/lose count 3.9000000000000004/78.20000000000003 (-74.30000000000003)
Epoch 007/150 | Loss 0.0404 | Win/lose count 3.5000000000000004/83.40000000000012 (-79.90000000000012)
Epoch 008/150 | Loss 0.0449 | Win/lose count 3.700000000000001/79.40000000000006 (-75.70000000000006)
Epoch 009/150 | Loss 0.0766 | Win/lose count 1.0999999999999996/88.00000000000017 (-86.90000000000018)
Epoch 010/150 | 

Epoch 083/150 | Loss 0.0355 | Win/lose count 1.7000000000000002/79.60000000000007 (-77.90000000000006)
Epoch 084/150 | Loss 0.0224 | Win/lose count 2.0000000000000004/83.4000000000001 (-81.4000000000001)
Epoch 085/150 | Loss 0.0341 | Win/lose count 2.9000000000000012/88.80000000000015 (-85.90000000000015)
Epoch 086/150 | Loss 0.0325 | Win/lose count 1.9000000000000004/86.80000000000013 (-84.90000000000012)
Epoch 087/150 | Loss 0.0387 | Win/lose count 0.9999999999999998/90.40000000000013 (-89.40000000000013)
Epoch 088/150 | Loss 0.0434 | Win/lose count 1.5/90.40000000000025 (-88.90000000000025)
Epoch 089/150 | Loss 0.0444 | Win/lose count 1.5/87.40000000000016 (-85.90000000000016)
Epoch 090/150 | Loss 0.0407 | Win/lose count 2.600000000000001/83.00000000000004 (-80.40000000000005)
Epoch 091/150 | Loss 0.0825 | Win/lose count 1.0999999999999996/99.00000000000024 (-97.90000000000025)
Epoch 092/150 | Loss 0.0394 | Win/lose count 1.2999999999999998/83.2000000000001 (-81.9000000000001)
Epoch

In [73]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train= False)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # No reinforcement strategy for a random agent
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [74]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 1.5/0. Average score (1.5)
Win/lose count 2.0/0. Average score (1.75)
Win/lose count 3.5/0. Average score (2.3333333333333335)
Win/lose count 4.0/0. Average score (2.75)
Win/lose count 0.5/0. Average score (2.3)
Win/lose count 7.0/0. Average score (3.0833333333333335)
Win/lose count 3.0/0. Average score (3.0714285714285716)
Win/lose count 0.5/0. Average score (2.75)
Win/lose count 4.5/0. Average score (2.9444444444444446)
Win/lose count 6.5/0. Average score (3.3)
Win/lose count 0/0. Average score (3.0)
Win/lose count 5.0/0. Average score (3.1666666666666665)
Win/lose count 7.0/0. Average score (3.4615384615384617)
Win/lose count 2.5/0. Average score (3.392857142857143)
Win/lose count 0.5/0. Average score (3.2)
Win/lose count 5.0/0. Average score (3.3125)
Win/lose count 3.0/0. Average score (3.2941176470588234)
Win/lose count 3.0/0. Average score (3.2777777777777777)
Win/lose count 4.0/1.0. Average score (3.263157894736842)
Win/lose count 2.5/0. Average score (3.225)
Win/

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***